<a href="https://colab.research.google.com/github/dowrave/PythonToKaggle/blob/main/3_NewYorkTaxiFarePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle/
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c new-york-city-taxi-fare-prediction # 아니 ㄹㅇ 책이 검수가 덜 됨
!unzip new-york-city-taxi-fare-prediction.zip

In [ ]:
# 데이터 파일 크기 확인 함수
import os
def convert_bytes(file_path, unit = None):
  size = os.path.getsize(file_path)
  if unit == "KB":
    return print("File size :" , str(round(size/1024, 3)), "KB")
  elif unit == "MB":
    return print("File size :" , str(round(size/1024**2, 3)), "MB")
  elif unit == "GB":
    return print("File size :" , str(round(size/1024**3, 3)), "GB")
  else:
    return print("File size :" , str(size/1024), "Bytes")

  
file_path = 'train.csv'
convert_bytes(file_path)
convert_bytes(file_path, 'GB')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def skip_logic(index, skip_num):
  if index % skip_num == 0:
    return False
  return True 

# 이 과정은 한나절 걸림. 6gb이기 때문.
train = pd.read_csv('train.csv', skiprows = lambda x: skip_logic(x, 4), parse_dates = ['pickup_datetime'])
print(train.shape)
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

## 지도 학습에서 가장 먼저 시각화 할 것은 종속 변수 Y이다
- 수치 데이터라면 정규 분포를 이루는지 확인하는 것이 기초 통계의 여러 통계 모델을 적용할 때 매우 중요하기 때문.


In [ ]:
print(train.info, '-' * 30, test.info)

In [ ]:
train.head()

In [ ]:
train.describe() # fare_amount 값은 음수가 나올 수 없는 값임 - 음수가 나온 데이터는 신뢰할 수 없는 데이터이니 삭제하는 게 좋음

In [ ]:
test.describe()

In [ ]:
sum(train['fare_amount'] < 0)

In [ ]:
# 음수의 개수 확인하기
from collections import Counter
Counter(train['fare_amount'] < 0 ) # 조건을 만족하는 값은 True에, 아니면 False에 들어간다

In [ ]:
# 음수 데이터 삭제
train = train.drop(train[train['fare_amount'] < 0].index, axis = 0).reset_index(drop = True)
Counter(train['fare_amount'] < 0 )

In [ ]:
# 그 외에도 지나치게 높은 값 (가령 최댓값은 9만 달러가 넘음 : 전산 오류인지 실제인지 알 도리가 없음) 제거
train = train.drop(train[train['fare_amount'] > 500].index, axis = 0 ).reset_index(drop = True)
print(train['fare_amount'].min())
print(train['fare_amount'].max())

In [ ]:
# 데이터가 1천만 개 정도 되므로, 모든 데이터에 대해 시각화 작업을 진행하는 것은 비용적인 측면에서 나쁘다
# 따라서 Stratified Sampling을 진행한다.
train['passenger_count'].unique()

In [ ]:
# 택시를 타는데 208, 0, 129, 51, 6, 8, 9 등의 값은 이상함(물론 6까지는 택시 자체가 특수한 종류의 차였을 수도 있겠다고 할 수는 있다)
# 어쨌든 책에서는 6을 초과하는 데이터는 제거함
train = train.drop(train[train['passenger_count'] > 6].index, axis = 0).reset_index(drop = True)
train['passenger_count'].unique()

In [ ]:
# 0도 그냥 같이 제거하지;
train = train.drop(train[train['passenger_count']  == 0].index, axis = 0).reset_index(drop = True)
train['passenger_count'].unique()

In [ ]:
# Passenger_Count의 비율 확인하기 (비율은 value_counts에 normalize = True 를 전달해서 만들 수 있다)
train['passenger_count'].value_counts(
    normalize = True
    ) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

""" Stratified 예제임 실제 코드 실행과는 관련 없는 부분"""
# sample = pd.DataFrame({'group' : np.repeat(['A', 'B', 'C'], (60, 40, 20)),
#                        'sample_value' : np.random.randn(120)})

# split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.7, random_state = 42)

# # strat_train_set = []
# # test_set = []

# for train_idx, test_idx in split.split(sample, sample['group']):
#   # 여기서 선언됐는데 잘 작동하네?
#   strata_train_set = sample.loc[train_idx]
#   test_set = sample.loc[test_idx]

# print(sample['group'].value_counts(normalize = True))
# print(sample.shape)
# print('-' * 40)
# print(strata_train_set['group'].value_counts(normalize = True))
# print(strata_train_set.shape)

In [ ]:
# 본 데이터에 적용 : 전체 표본의 0.1%의 데이터만 선택
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.001, random_state = 42)


for large_index, sample_index in split.split(train, train['passenger_count']):
  large_df = train.loc[large_index]
  sample_df = train.loc[sample_index]

print(train['passenger_count'].value_counts(normalize = True), train.shape)
print('-' * 40)
print(sample_df['passenger_count'].value_counts(normalize = True), sample_df.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(sample_df['fare_amount'])
plt.show()

- 그래프 해석 : 전형적인 왜도의 그래프. 0 ~ 25 $ 구간이 가장 많은 비율을 보인다
- 이런 데이터의 특성은 피쳐 엔지니어링 진행 시 로그 변환을 진행한다

### 위도, 경도 파악하기

In [ ]:
Counter(train['pickup_latitude'] < -90)

In [ ]:
Counter(train['pickup_latitude'] > 90)

In [ ]:
# 위도의 범위는 -90 ~ 90, 경도의 범위는 -180 ~ 180이다. 이외의 값은 모두 제거해주자.
train = train.drop(train[train['dropoff_latitude'] < - 90 ].index, axis = 0).reset_index(drop = True)
train = train.drop(train[train['dropoff_latitude'] > 90 ].index, axis = 0).reset_index(drop = True)
train = train.drop(train[train['pickup_longitude'] < - 180].index, axis = 0).reset_index(drop = True)
train = train.drop(train[train['pickup_longitude'] > 180 ].index, axis = 0).reset_index(drop = True)
train = train.drop(train[train['dropoff_longitude'] < - 180 ].index, axis = 0).reset_index(drop = True)
train = train.drop(train[train['dropoff_longitude'] > 180 ].index, axis = 0).reset_index(drop = True)

train.shape

In [ ]:
train.info()

In [ ]:
train.iloc[:3 , 0] # object이므로 date time으로 바꿔주자
# 책과 달리 pickup_datetime은 datetime으로 저장되어 있음
pd.to_datetime(train['key'], format = '%Y-%m-%d %H:%M:%S')


In [ ]:
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.001, random_state = 42) 
for large_index, sample_index in split.split(train, train['passenger_count']):
  print(len(sample_index))
  large_df = train.loc[large_index]
  sample_df = train.loc[sample_index] 

In [ ]:
print(large_df.shape)
print(sample_df.shape)

In [ ]:
# 위도, 경도를 가져오면 지도 위에 시각화를 할 수 있다.
# folium으로 가능하지만 여기서는 matplotlib을 통해 이미지 위에 점을 그릴 것

Bounding_Box = (-74.5, -72.8, 40.5, 41.8)
Bounding_Box_Zoom = (-74.3, -73.7, 40.5, 40.9)

# 지도의 범위 내에 있는 데이터만 추리는 함수
def select_within_boundingbox(data, BB):
  return (data.pickup_longitude >= BB[0]) & (data.pickup_longitude <= BB[1]) & \
        (data.pickup_latitude >= BB[2]) & (data.pickup_latitude <= BB[3]) & \
        (data.dropoff_longitude >= BB[0]) & (data.dropoff_longitude <= BB[1]) & \
        (data.dropoff_latitude >= BB[2]) & (data.dropoff_latitude <= BB[3])

In [ ]:
print("Old Size : %d"%len(sample_df))
sample_df = sample_df[select_within_boundingbox(sample_df, Bounding_Box)]
print("New Size : %d"%len(sample_df))

In [ ]:
# 이미지 불러오기
import ssl
from urllib.request import urlopen
context = ssl._create_unverified_context() # 웹 크롤링 시 자주 활용되는 함수

Bounding_Box = (-74.5, -72.8, 40.5, 41.8)
NYC_MAP_img_path = "https://aiblog.nl/download/nyc_-74.5_-72.8_40.5_41.8.png"
NYC_MAP = urlopen(NYC_MAP_img_path, context = context)
nyc_map = plt.imread(NYC_MAP)


Bounding_Box_Zoom = (-74.3, -73.7, 40.5, 40.9)
NYC_MAP_ZOOM_img_path = "https://aiblog.nl/download/nyc_-74.3_-73.7_40.5_40.9.png"
NYC_MAP_ZOOM = urlopen(NYC_MAP_ZOOM_img_path, context = context)
nyc_map_zoom = plt.imread(NYC_MAP_ZOOM)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16, 10))
ax[0].imshow(nyc_map, zorder=0 ,extent = Bounding_Box)
ax[0].set_title("NY Map")

ax[1].imshow(nyc_map_zoom, zorder = 0, extent = Bounding_Box_Zoom)
ax[1].set_title("NY Map Zoom")

plt.show()

In [ ]:
# 위에 산점도 그리기
def scatter_plot_on_map(df, BB, nyc_map, s=10, alpha = 0.2):
  fig,ax = plt.subplots(1, 2, figsize=(16,10))

  # Scatter 1
  ax[0].scatter(df.pickup_longitude, df.pickup_latitude, zorder = 1, alpha = alpha, c = 'r', s = s)
  ax[0].set_xlim(BB[0], BB[1])
  ax[0].set_ylim(BB[2], BB[3])
  ax[0].set_title('Pickup Locations')
  ax[0].imshow(nyc_map, zorder=0, extent = BB)

  # Scatter 2
  ax[1].scatter(df.dropoff_longitude, df.dropoff_latitude, zorder = 1, alpha = alpha, c = 'r', s = s)
  ax[1].set_xlim(BB[0], BB[1])
  ax[1].set_ylim(BB[2], BB[3])
  ax[1].set_title('Dropoff Locations')
  ax[1].imshow(nyc_map, zorder=0, extent = BB)

scatter_plot_on_map(sample_df, Bounding_Box, nyc_map, s=1, alpha = 0.3)

In [ ]:
scatter_plot_on_map(sample_df, Bounding_Box_Zoom, nyc_map, s=1, alpha = 0.3)


In [ ]:
# 날짜 데이터 빼내기
sample_df['Year'] = sample_df['pickup_datetime'].dt.year
sample_df['Month'] = sample_df['pickup_datetime'].dt.month
sample_df['Date'] = sample_df['pickup_datetime'].dt.day
sample_df['Day of Week'] = sample_df['pickup_datetime'].dt.dayofweek # 0 : 월 ~ 6 : 일
sample_df['Hour'] = sample_df['pickup_datetime'].dt.hour
print(sample_df.iloc[:, 8:])

In [ ]:
sample_df.head()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,6))
sns.histplot(sample_df['Day of Week'], ax= ax[0])
ax[0].set_xlabel("Day of Week")
ax[0].set_ylabel("Count")

sns.boxplot(x='Day of Week', y='fare_amount', data = sample_df, ax = ax[1])
ax[1].set_xlabel('Day of Week')

fig.tight_layout()
plt.show()

- 목, 금, 토에 이용객이 많음
- 아래 BoxPlot의 경우 log scale로 변환하면 더 쉽게 비교할 수 있으나 책에서는 생략되었음

### 시간대별 금액 확인

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
sns.barplot(x='Hour', y='fare_amount', data = sample_df, 
            estimator = sum, ax = ax[0]) # 디폴트 값은 mean임
ax[0].set_xlabel('Hours')
ax[0].set_ylabel('Sum : Fare Amount $')

sns.barplot(x='Hour', y='fare_amount', data = sample_df, 
            ax = ax[1]) 
ax[1].set_xlabel('Hours')
ax[1].set_ylabel('Mean : Fare Amount $')

plt.show()

## 피쳐 엔지니어링
- 위에서 한 과정 요약해서 함수로 만듦 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def skip_logic(index, skip_num):
  if index % skip_num == 0:
    return False
  return True

train = pd.read_csv('./train.csv', skiprows = lambda x: skip_logic(x, 1000))
print(train.shape)
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# 위도, 경도의 최소 / 최댓값은 그냥 이걸로 확인하고 넘길게요~
test.describe()

In [ ]:
def preprocessing(data):
  print("old data shape : ", data.shape)

  # 결측치 처리
  data = data.drop(data[data.isnull().any(1)].index, axis = 0)

  # 이상치 처리
  if 'fare_amount' in data.columns: # 테스트 데이터에는 fare_amount 가 없다
    data = data.drop(data[data['fare_amount'] < 0].index, axis = 0 ).reset_index(drop = True)
    data = data.drop(data[data['fare_amount'] > 500].index, axis = 0).reset_index(drop = True)

  # 지도의 범위를 넘어간다면 데이터 제거
  boundary = {'min_lng' : -74.26342, 'min_lat' : 40.573143, 'max_lng' : -72.986532, 'max_lat' : 41.709555}

  # ~는 Not을 의미함 : 조건을 모두 만족한다면 0, 하나라도 삐끗하면 1임

  data.loc[~((data.pickup_longitude >= boundary['min_lng']) & (data.pickup_longitude <= boundary['max_lng']) &
             (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
             (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
             (data.dropoff_latitude >= boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])), 'is_beyond_NY'] = 1

  data.loc[((data.pickup_longitude >= boundary['min_lng']) & (data.pickup_longitude <= boundary['max_lng']) &
             (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
             (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
             (data.dropoff_latitude >= boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])), 'is_beyond_NY'] = 0

  print(data['is_beyond_NY'].value_counts())

  data = data.drop(data[data['passenger_count'] == 1].index, axis = 0 ).reset_index(drop = True)
  data.drop(['is_beyond_NY'], axis = 1 , inplace = True)
  data = data.drop(data[data['passenger_count'] > 6].index, axis = 0 ).reset_index(drop = True)
  data = data.drop(data[data['passenger_count'] == 0].index, axis = 0 ).reset_index(drop = True)
  data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S UTC')

  # 날짜에서 변수 뽑아내기
  data['year'] = data['pickup_datetime'].dt.year
  data['month'] = data['pickup_datetime'].dt.month
  data['date'] = data['pickup_datetime'].dt.day
  data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
  data['hour'] = data['pickup_datetime'].dt.hour

  # 변수 제거
  data.drop(['key', 'pickup_datetime'], axis = 1, inplace = True)

  print("new data shape : ", data.shape)

  return data

new_train = preprocessing(train)
new_train

### 문제점!
- 이 지역에 대한 배경 도메인이 없는 경우 특징 추출을 어떻게 할 것인가?
  - 이 경우, 위도와 경도에 대한 지식을 거리로 통합해버림
  - 근데 위도와 경도를 거리로 어떻게 통합하느냐?
    - 아크하버사인(Arcsin)이라는 공식이 있음. 파이썬엔 haversine이라는 라이브러리 구현됨 (코랩에 없기 때문에 인스톨해야함)

In [ ]:
# !pip install haversine

In [ ]:
# 예제 : 서울 - 부산 간의 거리
from haversine import haversine, Unit 
seoul = (37.532700, 127.024612)
busan = (35.114839, 129.041494)

haversine(seoul, busan)

In [ ]:
# 본 데이터에 적용 : Pandas 내부 변수를 list로 변환한 뒤에 반복문을 적용해 하나씩 계산함
# ? : numpy로 하면 안되나?
distances = []
pick_lon = new_train['pickup_longitude'].tolist()
pick_lat = new_train['pickup_latitude'].tolist()
drop_lon = new_train['dropoff_longitude'].tolist()
drop_lat = new_train['dropoff_latitude'].tolist()

for row in range(len(new_train)):
  dist = haversine((pick_lat[row], pick_lon[row]), (drop_lat[row], drop_lon[row]))
  distances.append(dist)

new_train['distance'] = distances
print(new_train[['fare_amount', 'distance', 'passenger_count']])

In [ ]:
# 물론 이 값들은 직선 거리라서 실제 거리는 조금 더 길겠지만 데이터에서는 실제 주행거리를 제공하지 않기 때문에 이 정도가 한계임
# 사용된 위도, 경도 값을 제거
new_train.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis = 1, inplace = True)
new_train.info()

- 추가로 진행되어야 하는 과정들이지만 여기선 생략됨:
  - 수치형 데이터의 로그 변환, 표준화 / 정규화
  - day_of_week은 문자열데이터로 취급되어야 함(Categories?)
  - Label Encoding, One-Hot Encoding 등도 필요
  - [중요 : 도메인]
    - 택시가 가장 많이 활용되는 지역을 찾아서 피처 엔지니어링(캐글 노트북 에제 중에는 공항과 연계된 피쳐 엔지니어링이 있는 노트북을 찾을 수 있음)


## 모델링

#### 저자는 초보자에게 하이퍼파라미터 튜닝을 권하지 않음
- 투입 대비 결과물의 향상이 그렇게 높지 않을 확률이 높음
- 따라서 하이퍼파라미터 튜닝이 필요하다면, 캐글 그마 같은 사람들의 파라미터를 갖다 쓰는 것과 같이 효율적인 측면을 고려하는 게 더 중요할 수도 있음
- 하이퍼파라미터에 대한 논문 등을 읽어서 이론적 배경을 쌓는 것도 매우 훌륭한 공부일 수 있다

- 또한 하이퍼파라미터 튜닝을 맹신하면 생길 수 있는 문제점으로, 검증 세트에 대한 과적합이 있음
- 캐글에서 모덷 평가는 미지의 데이터가 그 대상임(실제로 2021년 2월에 종료된 Competition의 제출일은 2.22이지만 실제 예측 구간에 해당하는 날짜는 8.23임




## 하이퍼파라미터의 의미와 최적의 하이퍼파라미터 찾기

- Tree : Graphviz 패키지

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import graphviz

iris = load_iris()
X_features = iris.data
y_target = iris.target
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_features, y_target)
dot_data = tree.export_graphviz(clf, out_file = None,
                                feature_names = iris.feature_names,
                                class_names = iris.target_names, filled = True,
                                rounded = True, special_characters = True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
# clf 파라미터 출력
print(clf)

In [ ]:
# 트리 모델에서 Max_depth만 수정한 뒤 다시 실행
clf = tree.DecisionTreeClassifier(max_depth = 2)
clf = clf.fit(X_features, y_target)
dot_data = tree.export_graphviz(clf, out_file = None,
                                feature_names = iris.feature_names,
                                class_names = iris.target_names, filled = True,
                                rounded = True, special_characters = True)
graph = graphviz.Source(dot_data)
graph

### 그리드 서치 vs 랜덤 서치
- 랜덤 서치는 같은 비용으로 더 많은 탐색을 할 수 있다는 장점이 있다

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd

iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size = 0.2,
                                                    random_state = 42
                                                    )

# GridSearchCV
tree = DecisionTreeClassifier()
grid_serach_params = {'max_depth' : [1, 2],
                      'max_features' : [1, 2]}
grid_tree = GridSearchCV(tree, param_grid = grid_serach_params, cv = 5)
grid_tree.fit(X_train, y_train)
scores = pd.DataFrame(grid_tree.cv_results_)
scores[['params', 'mean_test_score', 'rank_test_score']]


In [ ]:
# RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

tree = DecisionTreeClassifier()
random_search_params = {'max_depth' : randint(1, 4),
                        'max_features' : randint(1, 4)}

random_tree = RandomizedSearchCV(tree, random_search_params, cv = 5)
random_tree.fit(X_train, y_train)

scores = pd.DataFrame(random_tree.cv_results_)
scores[['params', 'mean_test_score', 'rank_test_score']]

### 본 데이터에 적용
- XGBoost에는 Python API와 sklearn aPI가 같이 존재함. 그래서 같은 파라미터 이름도 다르게 쓰이는 경우가 종종 있음
  - ex) 학습률 : eta vs learning rate
    - 최소 손실(leaf 노드 추가로 나눌지 여부를 결정하는 값: 감마) - gamma vs Min_Split_Loss
    - 샘플링 비율 : Sub_Sample, Subsample
- 이는 LightGBM도 비슷함

### 따라서 사이킷런을 적용하느냐 독자적인 모듈을 쓸 거냐를 정하고 시작해야 하며
### 본문에서는 사이킷런을 적용함

In [ ]:
new_train.info()

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from math import sqrt 
import time
import datetime

y = new_train['fare_amount']
new_train.drop(['fare_amount'], axis = 1, inplace = True)
X = new_train
X.shape, y.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# XGBoost 파라미터 적용
params = {"max_depth" : 5,
          'learning_rate' : 0.5,
          'objective' : 'reg:linear', # 회귀 : 'reg:linear' / 이진 분류 : 'binary:logisitic' / 'multi:softmax' (개별 분류값 바로 확인) / 'multi : softprob' (개별)
          'eval_metric' : 'rmse',
          'early_stopping_rounds' : 5
          }
xgb_model = xgb.XGBRegressor(**params)
print(xgb_model)

start = time.time()
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
              verbose = True) # 검증 데이터의 학습 결과를 하나씩 출력해서 보여줌
sec = time.time() - start

times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
new_test = preprocessing(test)

distances = []
pick_lon = new_test['pickup_longitude'].tolist()
pick_lat = new_test['pickup_latitude'].tolist()
drop_lon = new_test['dropoff_longitude'].tolist()
drop_lat = new_test['dropoff_latitude'].tolist()

for row in range(len(new_test)):
  dist = haversine((pick_lat[row], pick_lon[row]), (drop_lat[row], drop_lon[row]))
  distances.append(dist)

new_test['distance'] = distances
new_test.drop(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis = 1 , inplace = True)


In [ ]:
new_test.columns

In [ ]:
y_pred_xgb = xgb_model.predict(new_test)
print(y_pred_xgb)

In [ ]:
submission['fare_amount'] = y_pred_xgb
submission.to_csv('final_submission.csv', index = False)

In [ ]:
# 자꾸 뭐가 쳐내지기 떄문에 그냥 코드만 작성하고 이번 과정은 종료하겠음
!kaggle competitions submit -c new-york-city-taxi-fare-prediction -f _final_submission.csv -m 'Message'

In [ ]:
# lightGBM

params = {"max_depth" : 5,
          'learning_rate' : 0.5,
          'objective' : 'regression', # 회귀 : 'reg:linear' / 이진 분류 : 'binary:logisitic' / 'multi:softmax' (개별 분류값 바로 확인) / 'multi : softprob' (개별)
          'eval_metric' : 'rmse',
          'early_stopping_rounds' : 5
          }


start = time.time()
lgb_model = lgb.LGBMRegressor(**params)
lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
              verbose = True) # 검증 데이터의 학습 결과를 하나씩 출력해서 보여줌
sec = time.time() - start

times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

y_pred_lgb = lgb_model.predict(new_test)
print(y_pred_xgb)

submission['fare_amount'] = y_pred_xgb
submission.to_csv('lightgbm_final_submission.csv', index = False)

!kaggle competitions submit -c new-york-city-taxi-fare-prediction -f lightgbm_final_submission.csv - m 'Message'

In [ ]:
# CatBoost

!pip install catboost

from catboost import CatBoostRegressor
params = {'depth' : 5,
          'learning_rate' : 0.5,
          'eval_metric' : 'RMSE',
          'early_stopping_rounds' : 5}

start = time.time()
cat_model = CatBoostRegressor(**params)
cat_model.fit(X_train, y_train, eval_set = (X_val, y_val), verbose = True)

sec = time.time()-start
times = str(datetime.timedelta(seconds = sec)).split(".")
times = times[0]
print(times)

y_pred_cat = cat_model.predict(new_test)
print(y_pred_cat)

submission['fare_Amount'] = y_pred_cat
submission.to_csv('catboost_final_submission.csv', index = False)

!kaggle competitions submit -c new-york-city-taxi-fare-prediction -f catboost_final_submission.csv -m "Message"